<h2 align='center'>Codebasics ML Course: Rank Order, KS Statistic Calculation Tutorial</h2>

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
pd.set_option('display.float_format', lambda x: '{:.2f}'.format(x))

df = pd.read_csv("data.csv")
df

,Borrower Name,Default Probability,Default Truth
0,Priya Rao,0.32,0
1,Raj Patel,0.67,1
2,Meera Gupta,0.56,1
3,Linda Johnson,0.18,0
4,Aditi Sharma,0.75,1
5,John Smith,0.49,1
6,Arjun Reddy,0.12,0
7,Emily Chen,0.44,0
8,Laura Kim,0.06,0
9,Vivek Singh,0.39,0


In [2]:
# Sort the dataframe by Default Probability in descending order
df_sorted = df.sort_values('Default Probability', ascending=False)
df_sorted

,Borrower Name,Default Probability,Default Truth
4,Aditi Sharma,0.75,1
1,Raj Patel,0.67,1
12,Anil Kumar,0.62,0
2,Meera Gupta,0.56,1
13,Sarah Lee,0.52,1
5,John Smith,0.49,1
7,Emily Chen,0.44,0
9,Vivek Singh,0.39,0
0,Priya Rao,0.32,0
10,Michael Brown,0.28,0


In [3]:
df_sorted['Quartile'] = pd.qcut(df_sorted['Default Probability'], q=4, labels=['Q1', 'Q2', 'Q3', 'Q4'])
df_sorted

,Borrower Name,Default Probability,Default Truth,Quartile
4,Aditi Sharma,0.75,1,Q4
1,Raj Patel,0.67,1,Q4
12,Anil Kumar,0.62,0,Q4
2,Meera Gupta,0.56,1,Q4
13,Sarah Lee,0.52,1,Q3
5,John Smith,0.49,1,Q3
7,Emily Chen,0.44,0,Q3
9,Vivek Singh,0.39,0,Q3
0,Priya Rao,0.32,0,Q2
10,Michael Brown,0.28,0,Q2


In [6]:
df_grouped = df_sorted.groupby('Quartile').apply(lambda x: pd.Series({
    'Minimum Probability': x['Default Probability'].min(),
    'Maximum Probability': x['Default Probability'].max(),
    'Events': x['Default Truth'].sum(),
    'Non-events': x['Default Truth'].count() - x['Default Truth'].sum(),    
}))
df_grouped.reset_index(inplace=True)
df_grouped

,Quartile,Minimum Probability,Maximum Probability,Events,Non-events
0,Q1,0.03,0.15,0.00,4.00
1,Q2,0.18,0.32,0.00,4.00
2,Q3,0.39,0.52,2.00,2.00
3,Q4,0.56,0.75,3.00,1.00


In [7]:
df_grouped.sort_values('Quartile', ascending=False, inplace=True)
df_grouped

,Quartile,Minimum Probability,Maximum Probability,Events,Non-events
3,Q4,0.56,0.75,3.00,1.00
2,Q3,0.39,0.52,2.00,2.00
1,Q2,0.18,0.32,0.00,4.00
0,Q1,0.03,0.15,0.00,4.00


In [8]:
df_grouped['Event Rate'] = df_grouped['Events']*100 / (df_grouped['Events']+df_grouped['Non-events'])
df_grouped['Non-event Rate'] = df_grouped['Non-events']*100 / (df_grouped['Events']+df_grouped['Non-events'])
df_grouped

,Quartile,Minimum Probability,Maximum Probability,Events,Non-events,Event Rate,Non-event Rate
3,Q4,0.56,0.75,3.00,1.00,75.00,25.00
2,Q3,0.39,0.52,2.00,2.00,50.00,50.00
1,Q2,0.18,0.32,0.00,4.00,0.00,100.00
0,Q1,0.03,0.15,0.00,4.00,0.00,100.00


In [9]:
df_grouped['Cum Events'] =  df_grouped['Events'].cumsum()
df_grouped['Cum Non-events'] =  df_grouped['Non-events'].cumsum()
df_grouped

,Quartile,Minimum Probability,Maximum Probability,Events,Non-events,Event Rate,Non-event Rate,Cum Events,Cum Non-events
3,Q4,0.56,0.75,3.00,1.00,75.00,25.00,3.00,1.00
2,Q3,0.39,0.52,2.00,2.00,50.00,50.00,5.00,3.00
1,Q2,0.18,0.32,0.00,4.00,0.00,100.00,5.00,7.00
0,Q1,0.03,0.15,0.00,4.00,0.00,100.00,5.00,11.00


In [10]:
df_grouped['Cum Event Rate'] = df_grouped['Cum Events'] * 100 / df_grouped['Events'].sum()
df_grouped['Cum Non-event Rate'] = df_grouped['Cum Non-events']*100 / df_grouped['Non-events'].sum()
df_grouped

,Quartile,Minimum Probability,Maximum Probability,Events,Non-events,Event Rate,Non-event Rate,Cum Events,Cum Non-events,Cum Event Rate,Cum Non-event Rate
3,Q4,0.56,0.75,3.00,1.00,75.00,25.00,3.00,1.00,60.00,9.09
2,Q3,0.39,0.52,2.00,2.00,50.00,50.00,5.00,3.00,100.00,27.27
1,Q2,0.18,0.32,0.00,4.00,0.00,100.00,5.00,7.00,100.00,63.64
0,Q1,0.03,0.15,0.00,4.00,0.00,100.00,5.00,11.00,100.00,100.00


In [11]:
df_grouped['KS'] = abs(df_grouped['Cum Event Rate'] - df_grouped['Cum Non-event Rate'])
df_grouped

,Quartile,Minimum Probability,Maximum Probability,Events,Non-events,Event Rate,Non-event Rate,Cum Events,Cum Non-events,Cum Event Rate,Cum Non-event Rate,KS
3,Q4,0.56,0.75,3.00,1.00,75.00,25.00,3.00,1.00,60.00,9.09,50.91
2,Q3,0.39,0.52,2.00,2.00,50.00,50.00,5.00,3.00,100.00,27.27,72.73
1,Q2,0.18,0.32,0.00,4.00,0.00,100.00,5.00,7.00,100.00,63.64,36.36
0,Q1,0.03,0.15,0.00,4.00,0.00,100.00,5.00,11.00,100.00,100.00,0.00
